# Generating a Feature Matrix with Dask

This notebook walks through an example of how to generate a feature matrix using Featuretools and an entityset created from Dask dataframes. This example uses the Home Credit Default Risk dataset which can be obtained from [Kaggle](https://www.kaggle.com/c/home-credit-default-risk/data).

Before running this notebook, you should download the data, and save all of the CSV files from the dataset into a directory called `data/homehome-credit-default-risk`. If you place the data in a different location you may need to update the code used to read in the CSV files so they can be found.

Set the `version` variable in the following cell to one of the keys in the primitive dictionary that follows to select a set of primitives to use. This selection will also set the number of workers in the Dask client as well as the blocksize used to create the Dask dataframes.

In [1]:
version = "v1"

In [2]:
# Dict: {version_key : ([trans_primitives], [agg_primtivies], num_workers, blocksize)}
primitive_dict = {
    "v1": (["and"], ["sum", "max"], 4, "40MB"), # 937 features
    "v2": (["and"], ["sum", "max", "min", "mean"], 1, "100MB"),  # 1545 features
    "v3": (["and", "add_numeric", "negate"], [], 4, "100MB"),  #5946 features
    "v4": (["and", "negate"], ["sum", "max", "min", "mean", "count", "any", "all"], 1, "100MB"),  #2083 features
}

In [3]:
import math
from datetime import datetime

import dask.dataframe as dd
import numpy as np
import pandas as pd
from dask.distributed import Client

import featuretools as ft
import featuretools.variable_types as vtypes

from tqdm import tqdm

2020-05-07 15:47:34,200 featuretools - WARNING    Featuretools failed to load plugin nlp_primitives from library nlp_primitives.__init__. For a full stack trace, set logging to debug.


Create a Dask client with the correct number of workers for the primitives being used.

In [4]:
try:
    client.close()
except:
    pass
client = Client(n_workers=primitive_dict[version][2])
client

Client Scheduler: tcp://127.0.0.1:55189 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 17.18 GB


Read in the raw CSV datafiles and store them in Dask dataframes. The blocksize is set from the value contained in `primitive_dict` above. These values were found to work well on a MacBook Pro with 4 cores and 16GB of memory, but may need to be adjusted based on the available memory in your system.

In [5]:
%%time
blocksize = primitive_dict[version][3]
# Read in the datasets and replace the anomalous values
app_train = dd.read_csv('data/home-credit-default-risk/application_train.csv', blocksize=blocksize).replace({365243: np.nan})
app_test = dd.read_csv('data/home-credit-default-risk/application_test.csv', blocksize=blocksize).replace({365243: np.nan})
bureau = dd.read_csv('data/home-credit-default-risk/bureau.csv', blocksize=blocksize).replace({365243: np.nan})
bureau_balance = dd.read_csv('data/home-credit-default-risk/bureau_balance.csv', blocksize=blocksize).replace({365243: np.nan})
cash = dd.read_csv('data/home-credit-default-risk/POS_CASH_balance.csv', blocksize=blocksize).replace({365243: np.nan})
credit = dd.read_csv('data/home-credit-default-risk/credit_card_balance.csv', blocksize=blocksize).replace({365243: np.nan})
previous = dd.read_csv('data/home-credit-default-risk/previous_application.csv', blocksize=blocksize).replace({365243: np.nan})
installments = dd.read_csv('data/home-credit-default-risk/installments_payments.csv', blocksize=blocksize).replace({365243: np.nan})

CPU times: user 212 ms, sys: 16.7 ms, total: 229 ms
Wall time: 234 ms


Perform a few cleanup operations on the data.

In [6]:
%%time
app_test['TARGET'] = np.nan
app = app_train.append(app_test[app_train.columns])

for index in ['SK_ID_CURR', 'SK_ID_PREV', 'SK_ID_BUREAU']:
    for dataset in [app, bureau, bureau_balance, cash, credit, previous, installments]:
        if index in list(dataset.columns):
            dataset[index] = dataset[index].fillna(0).astype(np.int64)

es = ft.EntitySet(id='clients')

installments = installments.drop(columns=['SK_ID_CURR'])
credit = credit.drop(columns=['SK_ID_CURR'])
cash = cash.drop(columns=['SK_ID_CURR'])

CPU times: user 262 ms, sys: 7.18 ms, total: 270 ms
Wall time: 265 ms


The current implementation of Dask entities does not support inferring variable types. As a result, the user must specify the proper feature tools variable types for all of the columns in the dataframes used to create the entities. The following cell sets the appropriate datatypes for our entities.

In [7]:
app_vtypes = {
    'SK_ID_CURR': ft.variable_types.variable.Index,
    'AMT_ANNUITY': ft.variable_types.variable.Numeric,
    'AMT_CREDIT': ft.variable_types.variable.Numeric,
    'AMT_GOODS_PRICE': ft.variable_types.variable.Numeric,
    'AMT_INCOME_TOTAL': ft.variable_types.variable.Numeric,
    'AMT_REQ_CREDIT_BUREAU_DAY': ft.variable_types.variable.Numeric,
    'AMT_REQ_CREDIT_BUREAU_HOUR': ft.variable_types.variable.Numeric,
    'AMT_REQ_CREDIT_BUREAU_MON': ft.variable_types.variable.Numeric,
    'AMT_REQ_CREDIT_BUREAU_QRT': ft.variable_types.variable.Numeric,
    'AMT_REQ_CREDIT_BUREAU_WEEK': ft.variable_types.variable.Numeric,
    'AMT_REQ_CREDIT_BUREAU_YEAR': ft.variable_types.variable.Numeric,
    'APARTMENTS_AVG': ft.variable_types.variable.Numeric,
    'APARTMENTS_MEDI': ft.variable_types.variable.Numeric,
    'APARTMENTS_MODE': ft.variable_types.variable.Numeric,
    'BASEMENTAREA_AVG': ft.variable_types.variable.Numeric,
    'BASEMENTAREA_MEDI': ft.variable_types.variable.Numeric,
    'BASEMENTAREA_MODE': ft.variable_types.variable.Numeric,
    'CNT_CHILDREN': ft.variable_types.variable.Numeric,
    'CNT_FAM_MEMBERS': ft.variable_types.variable.Numeric,
    'CODE_GENDER': ft.variable_types.variable.Categorical,
    'COMMONAREA_AVG': ft.variable_types.variable.Numeric,
    'COMMONAREA_MEDI': ft.variable_types.variable.Numeric,
    'COMMONAREA_MODE': ft.variable_types.variable.Numeric,
    'DAYS_BIRTH': ft.variable_types.variable.Numeric,
    'DAYS_EMPLOYED': ft.variable_types.variable.Numeric,
    'DAYS_ID_PUBLISH': ft.variable_types.variable.Numeric,
    'DAYS_LAST_PHONE_CHANGE': ft.variable_types.variable.Numeric,
    'DAYS_REGISTRATION': ft.variable_types.variable.Numeric,
    'DEF_30_CNT_SOCIAL_CIRCLE': ft.variable_types.variable.Numeric,
    'DEF_60_CNT_SOCIAL_CIRCLE': ft.variable_types.variable.Numeric,
    'ELEVATORS_AVG': ft.variable_types.variable.Numeric,
    'ELEVATORS_MEDI': ft.variable_types.variable.Numeric,
    'ELEVATORS_MODE': ft.variable_types.variable.Numeric,
    'EMERGENCYSTATE_MODE': ft.variable_types.variable.Categorical,
    'ENTRANCES_AVG': ft.variable_types.variable.Numeric,
    'ENTRANCES_MEDI': ft.variable_types.variable.Numeric,
    'ENTRANCES_MODE': ft.variable_types.variable.Numeric,
    'EXT_SOURCE_1': ft.variable_types.variable.Numeric,
    'EXT_SOURCE_2': ft.variable_types.variable.Numeric,
    'EXT_SOURCE_3': ft.variable_types.variable.Numeric,
    'FLAG_CONT_MOBILE': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_10': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_11': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_12': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_13': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_14': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_15': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_16': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_17': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_18': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_19': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_2': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_20': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_21': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_3': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_4': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_5': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_6': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_7': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_8': ft.variable_types.variable.Boolean,
    'FLAG_DOCUMENT_9': ft.variable_types.variable.Boolean,
    'FLAG_EMAIL': ft.variable_types.variable.Boolean,
    'FLAG_EMP_PHONE': ft.variable_types.variable.Boolean,
    'FLAG_MOBIL': ft.variable_types.variable.Boolean,
    'FLAG_OWN_CAR': ft.variable_types.variable.Categorical,
    'FLAG_OWN_REALTY': ft.variable_types.variable.Categorical,
    'FLAG_PHONE': ft.variable_types.variable.Boolean,
    'FLAG_WORK_PHONE': ft.variable_types.variable.Boolean,
    'FLOORSMAX_AVG': ft.variable_types.variable.Numeric,
    'FLOORSMAX_MEDI': ft.variable_types.variable.Numeric,
    'FLOORSMAX_MODE': ft.variable_types.variable.Numeric,
    'FLOORSMIN_AVG': ft.variable_types.variable.Numeric,
    'FLOORSMIN_MEDI': ft.variable_types.variable.Numeric,
    'FLOORSMIN_MODE': ft.variable_types.variable.Numeric,
    'FONDKAPREMONT_MODE': ft.variable_types.variable.Categorical,
    'HOUR_APPR_PROCESS_START': ft.variable_types.variable.Numeric,
    'HOUSETYPE_MODE': ft.variable_types.variable.Categorical,
    'LANDAREA_AVG': ft.variable_types.variable.Numeric,
    'LANDAREA_MEDI': ft.variable_types.variable.Numeric,
    'LANDAREA_MODE': ft.variable_types.variable.Numeric,
    'LIVE_CITY_NOT_WORK_CITY': ft.variable_types.variable.Boolean,
    'LIVE_REGION_NOT_WORK_REGION': ft.variable_types.variable.Boolean,
    'LIVINGAPARTMENTS_AVG': ft.variable_types.variable.Numeric,
    'LIVINGAPARTMENTS_MEDI': ft.variable_types.variable.Numeric,
    'LIVINGAPARTMENTS_MODE': ft.variable_types.variable.Numeric,
    'LIVINGAREA_AVG': ft.variable_types.variable.Numeric,
    'LIVINGAREA_MEDI': ft.variable_types.variable.Numeric,
    'LIVINGAREA_MODE': ft.variable_types.variable.Numeric,
    'NAME_CONTRACT_TYPE': ft.variable_types.variable.Categorical,
    'NAME_EDUCATION_TYPE': ft.variable_types.variable.Categorical,
    'NAME_FAMILY_STATUS': ft.variable_types.variable.Categorical,
    'NAME_HOUSING_TYPE': ft.variable_types.variable.Categorical,
    'NAME_INCOME_TYPE': ft.variable_types.variable.Categorical,
    'NAME_TYPE_SUITE': ft.variable_types.variable.Categorical,
    'NONLIVINGAPARTMENTS_AVG': ft.variable_types.variable.Numeric,
    'NONLIVINGAPARTMENTS_MEDI': ft.variable_types.variable.Numeric,
    'NONLIVINGAPARTMENTS_MODE': ft.variable_types.variable.Numeric,
    'NONLIVINGAREA_AVG': ft.variable_types.variable.Numeric,
    'NONLIVINGAREA_MEDI': ft.variable_types.variable.Numeric,
    'NONLIVINGAREA_MODE': ft.variable_types.variable.Numeric,
    'OBS_30_CNT_SOCIAL_CIRCLE': ft.variable_types.variable.Numeric,
    'OBS_60_CNT_SOCIAL_CIRCLE': ft.variable_types.variable.Numeric,
    'OCCUPATION_TYPE': ft.variable_types.variable.Categorical,
    'ORGANIZATION_TYPE': ft.variable_types.variable.Categorical,
    'OWN_CAR_AGE': ft.variable_types.variable.Numeric,
    'REGION_POPULATION_RELATIVE': ft.variable_types.variable.Numeric,
    'REGION_RATING_CLIENT': ft.variable_types.variable.Numeric,
    'REGION_RATING_CLIENT_W_CITY': ft.variable_types.variable.Numeric,
    'REG_CITY_NOT_LIVE_CITY': ft.variable_types.variable.Boolean,
    'REG_CITY_NOT_WORK_CITY': ft.variable_types.variable.Boolean,
    'REG_REGION_NOT_LIVE_REGION': ft.variable_types.variable.Boolean,
    'REG_REGION_NOT_WORK_REGION': ft.variable_types.variable.Boolean,
    'TARGET': ft.variable_types.variable.Numeric,
    'TOTALAREA_MODE': ft.variable_types.variable.Numeric,
    'WALLSMATERIAL_MODE': ft.variable_types.variable.Categorical,
    'WEEKDAY_APPR_PROCESS_START': ft.variable_types.variable.Categorical,
    'YEARS_BEGINEXPLUATATION_AVG': ft.variable_types.variable.Numeric,
    'YEARS_BEGINEXPLUATATION_MEDI': ft.variable_types.variable.Numeric,
    'YEARS_BEGINEXPLUATATION_MODE': ft.variable_types.variable.Numeric,
    'YEARS_BUILD_AVG': ft.variable_types.variable.Numeric,
    'YEARS_BUILD_MEDI': ft.variable_types.variable.Numeric,
    'YEARS_BUILD_MODE': ft.variable_types.variable.Numeric
}

bureau_vtypes = {
    'SK_ID_BUREAU': ft.variable_types.variable.Index,
    'SK_ID_CURR': ft.variable_types.variable.Id,
    'CREDIT_ACTIVE': ft.variable_types.variable.Categorical,
    'CREDIT_CURRENCY': ft.variable_types.variable.Categorical,
    'DAYS_CREDIT': ft.variable_types.variable.Numeric,
    'CREDIT_DAY_OVERDUE': ft.variable_types.variable.Numeric,
    'DAYS_CREDIT_ENDDATE': ft.variable_types.variable.Numeric,
    'DAYS_ENDDATE_FACT': ft.variable_types.variable.Numeric,
    'AMT_CREDIT_MAX_OVERDUE': ft.variable_types.variable.Numeric,
    'CNT_CREDIT_PROLONG': ft.variable_types.variable.Numeric,
    'AMT_CREDIT_SUM': ft.variable_types.variable.Numeric,
    'AMT_CREDIT_SUM_DEBT': ft.variable_types.variable.Numeric,
    'AMT_CREDIT_SUM_LIMIT': ft.variable_types.variable.Numeric,
    'AMT_CREDIT_SUM_OVERDUE': ft.variable_types.variable.Numeric,
    'CREDIT_TYPE': ft.variable_types.variable.Categorical,
    'DAYS_CREDIT_UPDATE': ft.variable_types.variable.Numeric,
    'AMT_ANNUITY': ft.variable_types.variable.Numeric
}

previous_vtypes = {
    'SK_ID_PREV': ft.variable_types.variable.Index,
    'SK_ID_CURR': ft.variable_types.variable.Id,
    'NAME_CONTRACT_TYPE': ft.variable_types.variable.Categorical,
    'AMT_ANNUITY': ft.variable_types.variable.Numeric,
    'AMT_APPLICATION': ft.variable_types.variable.Numeric,
    'AMT_CREDIT': ft.variable_types.variable.Numeric,
    'AMT_DOWN_PAYMENT': ft.variable_types.variable.Numeric,
    'AMT_GOODS_PRICE': ft.variable_types.variable.Numeric,
    'WEEKDAY_APPR_PROCESS_START': ft.variable_types.variable.Categorical,
    'HOUR_APPR_PROCESS_START': ft.variable_types.variable.Numeric,
    'FLAG_LAST_APPL_PER_CONTRACT': ft.variable_types.variable.Categorical,
    'NFLAG_LAST_APPL_IN_DAY': ft.variable_types.variable.Boolean,
    'RATE_DOWN_PAYMENT': ft.variable_types.variable.Numeric,
    'RATE_INTEREST_PRIMARY': ft.variable_types.variable.Numeric,
    'RATE_INTEREST_PRIVILEGED': ft.variable_types.variable.Numeric,
    'NAME_CASH_LOAN_PURPOSE': ft.variable_types.variable.Categorical,
    'NAME_CONTRACT_STATUS': ft.variable_types.variable.Categorical,
    'DAYS_DECISION': ft.variable_types.variable.Numeric,
    'NAME_PAYMENT_TYPE': ft.variable_types.variable.Categorical,
    'CODE_REJECT_REASON': ft.variable_types.variable.Categorical,
    'NAME_TYPE_SUITE': ft.variable_types.variable.Categorical,
    'NAME_CLIENT_TYPE': ft.variable_types.variable.Categorical,
    'NAME_GOODS_CATEGORY': ft.variable_types.variable.Categorical,
    'NAME_PORTFOLIO': ft.variable_types.variable.Categorical,
    'NAME_PRODUCT_TYPE': ft.variable_types.variable.Categorical,
    'CHANNEL_TYPE': ft.variable_types.variable.Categorical,
    'SELLERPLACE_AREA': ft.variable_types.variable.Numeric,
    'NAME_SELLER_INDUSTRY': ft.variable_types.variable.Categorical,
    'CNT_PAYMENT': ft.variable_types.variable.Numeric,
    'NAME_YIELD_GROUP': ft.variable_types.variable.Categorical,
    'PRODUCT_COMBINATION': ft.variable_types.variable.Categorical,
    'DAYS_FIRST_DRAWING': ft.variable_types.variable.Numeric,
    'DAYS_FIRST_DUE': ft.variable_types.variable.Numeric,
    'DAYS_LAST_DUE_1ST_VERSION': ft.variable_types.variable.Numeric,
    'DAYS_LAST_DUE': ft.variable_types.variable.Numeric,
    'DAYS_TERMINATION': ft.variable_types.variable.Numeric,
    'NFLAG_INSURED_ON_APPROVAL': ft.variable_types.variable.Numeric
}

bureau_balance_vtypes = {
    'bureaubalance_index': ft.variable_types.variable.Index,
    'SK_ID_BUREAU': ft.variable_types.variable.Id,
    'MONTHS_BALANCE': ft.variable_types.variable.Numeric,
    'STATUS': ft.variable_types.variable.Categorical
}

cash_vtypes = {
    'cash_index': ft.variable_types.variable.Index,
    'SK_ID_PREV': ft.variable_types.variable.Id,
    'MONTHS_BALANCE': ft.variable_types.variable.Numeric,
    'CNT_INSTALMENT': ft.variable_types.variable.Numeric,
    'CNT_INSTALMENT_FUTURE': ft.variable_types.variable.Numeric,
    'NAME_CONTRACT_STATUS': ft.variable_types.variable.Categorical,
    'SK_DPD': ft.variable_types.variable.Numeric,
    'SK_DPD_DEF': ft.variable_types.variable.Numeric
}

installments_vtypes = {
    'installments_index': ft.variable_types.variable.Index,
    'SK_ID_PREV': ft.variable_types.variable.Id,
    'NUM_INSTALMENT_VERSION': ft.variable_types.variable.Numeric,
    'NUM_INSTALMENT_NUMBER': ft.variable_types.variable.Numeric,
    'DAYS_INSTALMENT': ft.variable_types.variable.Numeric,
    'DAYS_ENTRY_PAYMENT': ft.variable_types.variable.Numeric,
    'AMT_INSTALMENT': ft.variable_types.variable.Numeric,
    'AMT_PAYMENT': ft.variable_types.variable.Numeric
}

credit_vtypes = {
    'credit_index': ft.variable_types.variable.Index,
    'SK_ID_PREV': ft.variable_types.variable.Id,
    'MONTHS_BALANCE': ft.variable_types.variable.Numeric,
    'AMT_BALANCE': ft.variable_types.variable.Numeric,
    'AMT_CREDIT_LIMIT_ACTUAL': ft.variable_types.variable.Numeric,
    'AMT_DRAWINGS_ATM_CURRENT': ft.variable_types.variable.Numeric,
    'AMT_DRAWINGS_CURRENT': ft.variable_types.variable.Numeric,
    'AMT_DRAWINGS_OTHER_CURRENT': ft.variable_types.variable.Numeric,
    'AMT_DRAWINGS_POS_CURRENT': ft.variable_types.variable.Numeric,
    'AMT_INST_MIN_REGULARITY': ft.variable_types.variable.Numeric,
    'AMT_PAYMENT_CURRENT': ft.variable_types.variable.Numeric,
    'AMT_PAYMENT_TOTAL_CURRENT': ft.variable_types.variable.Numeric,
    'AMT_RECEIVABLE_PRINCIPAL': ft.variable_types.variable.Numeric,
    'AMT_RECIVABLE': ft.variable_types.variable.Numeric,
    'AMT_TOTAL_RECEIVABLE': ft.variable_types.variable.Numeric,
    'CNT_DRAWINGS_ATM_CURRENT': ft.variable_types.variable.Numeric,
    'CNT_DRAWINGS_CURRENT': ft.variable_types.variable.Numeric,
    'CNT_DRAWINGS_OTHER_CURRENT': ft.variable_types.variable.Numeric,
    'CNT_DRAWINGS_POS_CURRENT': ft.variable_types.variable.Numeric,
    'CNT_INSTALMENT_MATURE_CUM': ft.variable_types.variable.Numeric,
    'NAME_CONTRACT_STATUS': ft.variable_types.variable.Categorical,
    'SK_DPD': ft.variable_types.variable.Numeric,
    'SK_DPD_DEF': ft.variable_types.variable.Numeric
}

Create the entityset using the previously created Dask dataframes. The process for creating entities and entitysets from Dask dataframes is the same as creating entities and entitysets from pandas dataframes, with the one exception being that the `variable_types` parameter must be used when creating entities from Dask dataframes.

In [8]:
%%time
# Entities with a unique index
es = es.entity_from_dataframe(entity_id='app', dataframe=app, index='SK_ID_CURR',
                              variable_types=app_vtypes)

es = es.entity_from_dataframe(entity_id='bureau', dataframe=bureau, index='SK_ID_BUREAU',
                              variable_types=bureau_vtypes)

es = es.entity_from_dataframe(entity_id='previous', dataframe=previous, index='SK_ID_PREV',
                              variable_types=previous_vtypes)

# Entities that do not have a unique index
es = es.entity_from_dataframe(entity_id='bureau_balance', dataframe=bureau_balance,
                              make_index=True, index='bureaubalance_index',
                              variable_types=bureau_balance_vtypes)

es = es.entity_from_dataframe(entity_id='cash', dataframe=cash,
                              make_index=True, index='cash_index',
                              variable_types=cash_vtypes)

es = es.entity_from_dataframe(entity_id='installments', dataframe=installments,
                              make_index=True, index='installments_index',
                              variable_types=installments_vtypes)

es = es.entity_from_dataframe(entity_id='credit', dataframe=credit,
                              make_index=True, index='credit_index',
                              variable_types=credit_vtypes)

print("Adding relationships...")
# Relationship between app_train and bureau
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])

# Relationship between bureau and bureau balance
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])

# Relationship between current app and previous apps
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])

# Relationships between previous apps and cash, installments, and credit
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

# Add in the defined relationships
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])
# Print out the EntitySet
print(es)

Adding relationships...
Entityset: clients
  Entities:
    app [Rows: Delayed('int-a517f98c-8742-480c-a6ef-298417251af9'), Columns: 122]
    bureau [Rows: Delayed('int-de9237cf-b8c8-4c8d-9774-83344040b6ff'), Columns: 17]
    previous [Rows: Delayed('int-356cd0a8-8f8f-4eb8-a2f9-0813e86372e8'), Columns: 37]
    bureau_balance [Rows: Delayed('int-2711e447-0f80-464c-9a2c-f4a83ea82ae1'), Columns: 4]
    cash [Rows: Delayed('int-6b7bfcd1-3ba6-43f4-b4ba-58b5fddda54e'), Columns: 8]
    installments [Rows: Delayed('int-95b3b938-70e6-4437-ae3f-093e7f2e3b08'), Columns: 8]
    credit [Rows: Delayed('int-caa5b6f8-8bba-4647-b61d-461deb233504'), Columns: 23]
  Relationships:
    bureau.SK_ID_CURR -> app.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV
CPU times: user 4.06 s, sys: 411 ms, total: 4.47 s
Wall

Create a `cutoff_times` dataframe to supply to `ft.dfs()`. This step is optional and the `cutoff_time` parameter can be omitted when calling DFS. With the current implementation, supplying a cutoff time dataframe is slightly faster than supplying a single cutoff time value, although both approaches should work equivalently. This is something that should be improved in future updates of Featuretools.

In [9]:
%%time
cutoff_times = app['SK_ID_CURR'].to_frame().rename(columns={"SK_ID_CURR":"instance_id"})
cutoff_times["time"] = datetime.now()
cutoff_times = cutoff_times.compute()

CPU times: user 176 ms, sys: 24.2 ms, total: 200 ms
Wall time: 2.5 s


Run `ft_dfs()` to generate a list of features. We will use this list below to repartition our target entity to help avoid memory issues.

In [10]:
%%time
trans_primitives = primitive_dict[version][0]
agg_primitives = primitive_dict[version][1]

print("Running DFS...")
features = ft.dfs(entityset=es, target_entity='app',
                  trans_primitives=trans_primitives,
                  agg_primitives=agg_primitives,
                  where_primitives=[], seed_features=[],
                  max_depth=2, verbose=1, features_only=True,
                  cutoff_time=cutoff_times)

Running DFS...
Built 937 features
CPU times: user 1.09 s, sys: 23 ms, total: 1.12 s
Wall time: 1.1 s


The number of partitions in the feature matrix dataframe is currently determined from the number of partitions in the target entity. These partitions were set above when we created the dataframes used to create the entityset. For this example, our target entity is the `app` entity.

We expect to generate a large number of feature columns in our feature matrix relative to the number of columns in the `app` entity dataframe. The code below will repartition the `app` dataframe, which will, in turn, help us make sure that the partition size for the resulting feature matrix does not get too large, which could result in memory issues when attempting to save or compute the feature matrix.

In [11]:
%%time
new_partitions = es['app'].df.npartitions * math.ceil(len(features) / len(es['app'].df.columns))
print("New Partitions: {}".format(new_partitions))
es['app'].df = es['app'].df.repartition(npartitions=new_partitions)

New Partitions: 48
CPU times: user 1.07 ms, sys: 48 µs, total: 1.12 ms
Wall time: 1.1 ms


Now, we can run `ft.dfs()` to generate the feature matrix. The feature matrix will be returned as a Dask dataframe.

In [12]:
%%time
fm, features = ft.dfs(entityset=es, target_entity='app',
                      trans_primitives=trans_primitives,
                      agg_primitives=agg_primitives,
                      where_primitives=[], seed_features=[],
                      max_depth=2, verbose=0,
                      cutoff_time=cutoff_times)

CPU times: user 3.44 s, sys: 101 ms, total: 3.54 s
Wall time: 3.49 s


The feature matrix can now be saved to disk. Note, this process may take several minutes to complete, depending on the size of the feautre matrix that was generated.

In [13]:
%%time
fm.to_csv(f"dask-output-tmp/test_dask_{version}-*.csv", index=False)

Output feature matrix to CSV...
CPU times: user 25.8 s, sys: 2.05 s, total: 27.9 s
Wall time: 2min 34s


Alternatively, the feature matrix can be brought into memory by running `.compute()` on the Dask feature matrix returend from `ft.dfs()`. Note, this process may fail depending on the size of the feature matrix generated and the available system memory.

In [14]:
%%time
fm_computed = fm.compute()
print("Shape: {}".format(fm_computed.shape))
print("Memory: {} MB".format(fm_computed.memory_usage().sum() / 1000000))

Shape: (356255, 938)
Memory: 1439.2702 MB
CPU times: user 26.3 s, sys: 4.63 s, total: 30.9 s
Wall time: 1min 57s


In [15]:
fm_computed.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,MAX(credit.previous.DAYS_FIRST_DUE),MAX(credit.previous.AMT_GOODS_PRICE),MAX(credit.previous.AMT_DOWN_PAYMENT),MAX(credit.previous.CNT_PAYMENT),MAX(credit.previous.HOUR_APPR_PROCESS_START),MAX(credit.previous.RATE_DOWN_PAYMENT),MAX(credit.previous.DAYS_FIRST_DRAWING),MAX(credit.previous.NFLAG_INSURED_ON_APPROVAL),MAX(credit.previous.DAYS_LAST_DUE_1ST_VERSION),SK_ID_CURR
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100002
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100003
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100004
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,270000.0,NaN,0.0,15.0,NaN,NaN,0.0,NaN,100006
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100007


Now that we are finished, we can close our Dask client.

In [16]:
client.close()